In [ ]:
!pip install scikit-learn==1.0
!pip install xgboost==1.4.2
!pip install catboost==0.26.1
!pip install pandas==1.3.3
!pip install radiant-mlhub==0.3.0
!pip install rasterio==1.2.8
!pip install numpy==1.21.2
!pip install pathlib==1.0.1
!pip install tqdm==4.62.3
!pip install joblib==1.0.1
!pip install matplotlib==3.4.3
!pip install Pillow==8.3.2
!pip install torch==1.9.1
!pip install plotly==5.3.1


In [ ]:
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings('RuntimeWarning')

from radiant_mlhub import Collection
import tarfile
import os
from pathlib import Path
import json
from tqdm import tqdm
from joblib import Parallel,delayed
import datetime
import rasterio
import numpy as np
import pandas as pd

import gc

gc.collect()

0

In [ ]:
competition_train_df = pd.read_csv('train_data_sentinel2.csv')

In [ ]:
def get_date_format(month,day):
    '''
        Structures the dates in a particular format
    '''
    if (str(month)=='nan') or (str(day)=='nan'):
        return 'nan'
    else:
        if month>=10:
            if day>=10:
                return f'month_{str(int(month))}_day_{str(int(day))}'
            else:
                return f'month_{str(int(month))}_day_0{str(int(day))}'
        else:
            if day>=10:
                return f'month_0{str(int(month))}_day_{str(int(day))}'
            else:
                return f'month_0{str(int(month))}_day_0{str(int(day))}'
    

In [ ]:
competition_train_df['month'] = pd.to_datetime(competition_train_df['datetime']).dt.month.values 
competition_train_df['day']   = pd.to_datetime(competition_train_df['datetime']).dt.day.values

competition_train_df['dates'] = competition_train_df.apply(lambda z: get_date_format(z['month'],z['day']),axis=1)

unique_dates = competition_train_df['dates'].unique()
unique_dates = np.array([z for z in unique_dates if 'nan' not in z])

print(f'Length of unique dates {len(unique_dates)}')

Length of unique dates 76


In [ ]:
date_dict = dict(zip(competition_train_df['datetime'].dropna().unique(),unique_dates))
date_dict = dict(sorted(date_dict.items(), key=lambda item: item[1]))
date_order_to_consider = np.array(list(date_dict.values()))
tile_ids_train = competition_train_df['tile_id'].unique()

In [ ]:
len(tile_ids_train)

2650

In [ ]:
def get_bands(tile_date_times,tile_df,band,date_dict):
    '''
        Getting band dictionary with dates
    '''
    X_tile    = np.zeros((256 * 256, 76))
    X_tile[:] = np.nan
    
    for date_time in tile_date_times:
        
        
        source   = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']==band)]['file_path'].values[0])

        ### Flattening the file to get a vector for the image 
        array    = np.expand_dims(source.read(1).flatten(), axis=1)

        ### Capturing the date at which we need to replace the vector
        val      = date_dict[date_time]

        ### index at which replacement is to be done
        indices  = np.where(date_order_to_consider==val)[0][0]            

        X_tile[:,indices]   = array.ravel()

    return X_tile

In [ ]:
def get_dataframe(data_dict_band,band,y,field_ids):
    
    X = np.array([values.tolist() for _,values in tqdm(data_dict_band.items())]).reshape(-1,76)
    colnames          = [band+'_'+z for z in date_order_to_consider]
    data              = pd.DataFrame(X,columns=colnames)
    data['label']     = y.astype(int)
    data['field_id']  = field_ids
    
    mean_df           = data.groupby('field_id').mean().reset_index()
    low_df            = data.groupby('field_id').quantile(0.25).reset_index()
    up_df             = data.groupby('field_id').quantile(0.75).reset_index()
    med_df            = data.groupby('field_id').median().reset_index()
    
    
    return mean_df,low_df,up_df,med_df

In [ ]:
lb = 0 
ub = 200

bands_available = ['B01','B02','B03','B04','B05','B06','B07','B08','B8A','B09','B11','B12']



for band in bands_available:

    for batch in tqdm(range(0,14)):
        print(f'Performing operations for batch {batch+1}/14 for band {band}')
        data_dict_band = {}
        

        count          = 1

        y              = np.empty((0, 1))


        field_ids      = np.empty((0, 1))


        for tile_id in tile_ids_train[lb+(batch*200):ub+(batch*200)]:
            if tile_id != '1951': # avoid using this specific tile for the Hackathon as it might have a missing file

                tile_df               = competition_train_df[competition_train_df['tile_id']==tile_id]

                label_src             = rasterio.open(tile_df[tile_df['asset']=='labels']['file_path'].values[0])
                label_array           = label_src.read(1)

                y                     = np.append(y, label_array.flatten())

                field_id_src          = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])

                field_id_array        = field_id_src.read(1)

                field_ids             = np.append(field_ids, field_id_array.flatten())

                tile_date_times       = tile_df[tile_df['satellite_platform']=='s2']['datetime'].unique()

                data_dict_band[count]  = get_bands(tile_date_times,tile_df,band,date_dict)
                


                count                 = count+1
        gc.collect()        

        mean_df,low_df,up_df,med_df = get_dataframe(data_dict_band,band,y,field_ids)
        
        batchid           = int(batch)+1

        mean_df.to_csv(f'/root/sentinel2_updated/Train_data_prep/Batchwise_data/{band}/mean_{band}_{batchid}.csv',index=False)
#         low_df.to_csv(f'/root/sentinel2_updated/Train_data_prep/Batchwise_data/{band}/lq_{band}_{batchid}.csv',index=False)
#         up_df.to_csv(f'/root/sentinel2_updated/Train_data_prep/Batchwise_data/{band}/uq_{band}_{batchid}.csv',index=False)
        med_df.to_csv(f'/root/sentinel2_updated/Train_data_prep/Batchwise_data/{band}/median_{band}_{batchid}.csv',index=False)

        

        del field_ids,data_dict_band,label_src,field_id_src,label_array,field_id_array,y,mean_df,med_df,up_df,low_df
        del tile_df,tile_date_times
        gc.collect()
        gc.collect()
        gc.collect()

  0%|          | 0/14 [00:00<?, ?it/s]

Performing operations for batch 1/14 for band B01



  7%|▋         | 1/14 [07:39<1:39:38, 459.90s/it]

Performing operations for batch 2/14 for band B01



 14%|█▍        | 2/14 [15:24<1:32:30, 462.51s/it]

...


 86%|████████▌ | 12/14 [1:47:52<17:56, 538.06s/it]

Performing operations for batch 13/14 for band B12



 93%|█████████▎| 13/14 [1:56:43<08:55, 535.97s/it]

Performing operations for batch 14/14 for band B12



100%|██████████| 14/14 [1:58:44<00:00, 508.86s/it]
